In [6]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np
import cv2
import io
import logging
from flask_cors import CORS

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Load the trained model
try:
    model = tf.keras.models.load_model("data/skin_cancer_model.keras")
except Exception as e:
    logging.error(f"Error loading model: {e}")
    model = None

# Define class labels
class_labels = ["akiec", "bcc", "bkl", "df", "mel", "nv", "vasc"]

# Define recommendations
recommendations = {
    "akiec": "Consult a dermatologist immediately.",
    "bcc": "Requires medical attention; usually treated with surgery.",
    "bkl": "Often benign but needs monitoring.",
    "df": "Generally harmless, but consult a doctor for confirmation.",
    "mel": "Potentially dangerous, seek medical evaluation.",
    "nv": "Usually benign, but monitor for changes.",
    "vasc": "Generally harmless but can be checked by a doctor.",
}

def preprocess_image(image):
    """Preprocess the input image for model prediction."""
    try:
        image = cv2.imdecode(np.frombuffer(image.read(), np.uint8), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (128, 128))
        image = image / 255.0  # Normalize
        return np.expand_dims(image, axis=0)  # Add batch dimension
    except Exception as e:
        logging.error(f"Error preprocessing image: {e}")
        return None

@app.route("/predict", methods=["POST"])
def predict():
    """API endpoint to classify skin cancer from an uploaded image."""
    if model is None:
        return jsonify({"error": "Model not loaded"}), 500

    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]
    image = preprocess_image(file)

    if image is None:
        return jsonify({"error": "Invalid image format"}), 400

    try:
        prediction = model.predict(image)
        predicted_class = np.argmax(prediction)
        predicted_label = class_labels[predicted_class]
        recommendation = recommendations[predicted_label]

        response = {
            "cancer_type": predicted_label,
            "confidence": float(np.max(prediction)),  # Return confidence score
            "recommendation": recommendation,
        }
        return jsonify(response)
    
    except Exception as e:
        logging.error(f"Error making prediction: {e}")
        return jsonify({"error": "Prediction failed"}), 500

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


SystemExit: 1